This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [175]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [176]:
# First, import the relevant modules
import requests
import statistics
import json

In [177]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
quandl = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json', params={'api_key':API_KEY})

In [178]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(quandl.json()['dataset_data']['column_names'])
print(quandl.json()['dataset_data']['data'][0])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2019-09-24', 103.6, 105.2, 103.6, 105.0, None, 61623.0, 6452339.0, None, None, None]


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [179]:
# 1. Collect data from FSE for AFX_X in 2017
start_date = '2017-01-01'
end_date = '2017-12-31'
afx_x_2017 = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json', 
                      params={
                          'api_key':API_KEY, 
                          'start_date':start_date, 
                          'end_date':end_date
                      })

In [180]:
# 2. Convert afx_x_2017 to JSON python dictionary
afx_x_2017_dict = afx_x_2017.json()['dataset_data']
print(type(afx_x_2017_dict))

<class 'dict'>


In [181]:
# 3. Calculate highest and lowest opening prices

# Use a for loop to make the column names into keys of the dictionary
#     Initialize empty dict and alias column names
afx_dict = {}
names = afx_x_2017_dict['column_names']

#     Inner loop generates a list of values that correspond to each column name, 
#     Outer loop adds list as the value for each column name
for name in names:
    values = []
    iname = names.index(name)
    for row in afx_x_2017_dict['data']:
        values.append(row[iname])
    afx_dict[name] = values


# Calculate max and min of open. Ignore None values in the data.
highest_open = max([i for i in afx_dict['Open'] if i is not None])
lowest_open = min([i for i in afx_dict['Open'] if i is not None])

print(f'Highest open is ${highest_open:.2f}. \nLowest open is ${lowest_open:.2f}.', )

Highest open is $53.11. 
Lowest open is $34.00.


In [182]:
# 4. Calculate largest change in any one day

# The best solution would be use numpy arrays, but to avoid using 3rd party modules, use list comprehension
# to find the difference between the two columns
daily_changes = [a - b for a, b in zip(afx_dict['High'], afx_dict['Low'])]
largest_change = max(daily_changes)
print(f'The largest change is ${largest_change:.2f}.')

The largest change is $2.81.


In [183]:
# 5. Calculate the largest closing price change between any 2 days.

# Use a for loop to create a list of the differences between days, starting with the second day.
overnight_changes = []
for item in afx_dict['Close'][1:]:
    overnight_changes.append(item - afx_dict['Close'][afx_dict['Close'].index(item) - 1])

# Find the maximum overnight increase and decrease.
pos_max_overnight = max(overnight_changes)
neg_max_overnight = min(overnight_changes)

# Check if the largest difference is positive or negative and print the result.
if abs(neg_max_overnight) > pos_max_overnight:
    print(f'The largest overnight change is a ${neg_max_overnight:.2f} decrease.')
else:
    print(f'The largest overnight change is a ${abs(pos_max_overnight):.2f} increase.')



The largest overnight change is a $2.56 increase.


In [184]:
# 6. Find average daily trading volume.

# Use statistics module to calculate the mean.
avg_volume = statistics.mean(afx_dict['Traded Volume'])
print(f'The mean is {avg_volume:.2f} units.')

The mean is 89124.34 units.


In [185]:
# 7. Find median traded volume.

# Find the length of the Traded Volume column
median_ref = len(afx_dict['Traded Volume'])

# Created a sorted list of Traded Volumes
sorted_trade_vol = sorted(afx_dict['Traded Volume'])
    
# 
if median_ref % 2 == 0:
    median_index_1 = median_ref / 2 -1
    median_index_2 = median_index_1 + 1
    median_volume = (sorted_trade_vol[median_index_1] + sorted_trade_vol[median_index_2]) / 2
else:
    median_index = median_ref // 2 
    median_volume = sorted_trade_vol[median_index]
    
print(f'The median traded volume is {median_volume:.0f}.')


The median traded volume is 76286.
